In [1]:
 from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import tensorflow as tf

In [3]:
import numpy as np
import pandas as pd
import json

from tensorflow.python.keras.models import load_model

In [4]:
model = load_model('/content/gdrive/MyDrive/MachineLearningProject/TrainingModel')

In [5]:
with open('/content/gdrive/MyDrive/MachineLearningProject/tokenizer.json') as json_dosyasi:
    json_tokenizer = json.load(json_dosyasi)

In [7]:
def tokenlestir(yorumListesi):
    y_yorumlar = []
    for yorum in yorumListesi:
        y_yorum = []
        for kelime in yorum.split():
            if (len(y_yorum) < 50 and kelime in json_tokenizer):
                y_yorum.append(json_tokenizer[kelime])
                
        if (len(y_yorum) < 50):
            sifirlar = list(np.zeros(50 - len(y_yorum), dtype = int))
            y_yorum = sifirlar + y_yorum
            
        y_yorumlar.append(y_yorum)
    return np.array(y_yorumlar, dtype = np.dtype(np.int32))

In [14]:
text1 = "Great tablet for beginners! Very light and screen is wide. Enjoy using for travels.,Love it!"
text2 = "I bought this as a gift for my son and I know he will love it he has the old kindle fire as likes it a lot,Bought it for my son"
text3 = "This Kindle is so overloaded with apps that I will never use. Despite what one of Best Buys' associates said, you cannot remove any of the Amazon apps. The apps take up too much of the memory."
text4 = "Didn't want to go to the sites amazon wanted"
text5 = "Easy set up but did not meet my needs of being able to access Spectrum app."
text6 = "I am very happy with this Amazon tablet."

yazilar = [text1, text2, text3, text4, text5, text6]

deneme_kume = tokenlestir(yazilar)

sonuc = model.predict(deneme_kume)
sonuc

array([[0.9935616 ],
       [0.99379414],
       [0.01295397],
       [0.9339111 ],
       [0.03753102],
       [0.9871743 ]], dtype=float32)

In [16]:
from sklearn.metrics import confusion_matrix, classification_report


dataframe = pd.read_csv('/content/gdrive/MyDrive/MachineLearningProject/newDataset.csv', sep = ",")
target = dataframe['Puan']
data = dataframe['reviews.text'].astype(str)

target_int = [int(index) for index in target]

cutoff = int(len(data) * 0.85)
x_train, x_test = data[:cutoff], data[cutoff:]
y_train, y_test = target_int[:cutoff], target_int[cutoff:]


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [17]:
deneme_kume = tokenlestir(x_test)

sonuc = model.predict(deneme_kume)

sonuc = [round(index[0]) for index in sonuc]

cf_matrix = confusion_matrix(y_test, sonuc)

cf_matrix

array([[ 231,   64],
       [  26, 4878]])

In [24]:
falseCount = cf_matrix[0][1] + cf_matrix[1][0]

rate = 100 - (falseCount/len(sonuc)) * 100
rate

98.26889786497404